# 3 - Introducción a Tensorflow

**Sumario**

1. Introducción
2. Dispositivos de ejecución
3. Operaciones básicas
4. Cálculo de gradientes
5. Funciones
6. Operaciones matriciales

## 3.1 - Introducción

**TensorFlow (TF)** es una librería de software diseñada inicialmente por Google brain para facilitar el desarrollo de modelos basados en redes neuronales profundas. En adición a la libreria principal, hay dos versiones que permiten su despliegue en diferentes tipos de dispositivos y entornos:

* **TensorFlow Lite.** Conjunto de herramientas que ayudan a los desarrolladores a ejecutar modelos de TensorFlow en dispositivos incorporados, móviles o de IoT. Permite la inferencia de aprendizaje automático en dispositivos con una latencia baja y un tamaño de objeto binario reducido.
* **TensorFlow JS.** Biblioteca de JavaScript para el entrenamiento y la implementación de modelos de aprendizaje automático en navegadores y en Node.js.

In [1]:
import tensorflow as tf

print(tf.__version__)

2.11.0


## 3.2 - Dispositivos de ejecución

Una de las ventajas que nos ofrecen las librerías de desarrollo como TF es que podemos utilizar diferentes dispositivos de ejecución
1. CPU
2. GPU
3. TPU

Dependiendo del tipo de *hardware* que utilicemos, podremos incrementar la velocidad de nuestro proyecto de aprendizaje, si bien antes deberíamos comprobar cuáles de estos dispositivos se encuentran disponibles en nuestro entorno:

In [2]:
print(('Dispositivos disponibles: \n {0}').format([device.name for device in tf.config.experimental.list_physical_devices()]))

Dispositivos disponibles: 
 ['/physical_device:CPU:0']


De este modo, podemos utilizar diferentes dispositivos *hardware* a la hora de ejecutar nuestro código. Para ello, debemos seleccionar el dispositivo sobre el que queremos ejecutar utilizando `tf.device` e indicando el nombre del dispositivo que queremos utilizar.

In [3]:
import time
cpu_slot = 0
gpu_slot = 0

# Selecionamos el dispositivo hardware de tipo CPU
with tf.device('/CPU: ' + str(cpu_slot)):
    # Inicialización de un temporizador
    start = time.time()
    # Realización de una operación 
    for i in range(1, 10000):
        tf.eye(2,2)
    # Finalización del temporizador
    end = time.time() - start
    print (end)
    
# Selecionamos el dispositivo hardware de tipo GPU 
# with tf.device('/GPU: ' + str(cpu_slot)):
#     # Inicialización de un temporizador
#     start = time.time()
#     # Realización de una operación 
#     for i in range(1, 10000):
#         tf.eye(2,2)
#     # Finalización del temporizador
#     end = time.time() - start
#     print (end)

2.01548171043396


## 3.3 - Operaciones básicas

### 3.3.1 - Creación específica de tensores

Toda la información en TF se almacena mediante tensores que pueden representarse por medio de dos tipos de elementos:
1. constantes, que son variables inmutables durante la ejecución
2. variables, cuyo valor y formato pueden cambiar durante la ejecución
    
En el siguiente fragmento de código se describe como crear cada uno de estos dos tipos de elementos:

In [4]:
# Creación de una constante
C = tf.constant([12])
print(C)

# Creación de una variable
VA = tf.Variable([[1,2], [3,4]], name="VA")
print(VA)

tf.Tensor([12], shape=(1,), dtype=int32)
<tf.Variable 'VA:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>


### 3.3.2 - Creación general de tensores

La generación específica de tensores mediante las clases `tf.Variable` y `tf.constant` puede llegar a ser un tanto tediosa si tenemos que crear tensores de gran tamaño, así que vamos a ver cómo generar grandes matrices de forma mucho más sencilla por medio de tres funciones:
* `tf.zeros`, que permite construir tensores de diferente tamaño inicializando todas las variables a 0.
* `tf.ones`, que permite construir tensores de diferente tamaño inicializando todas las variables 1.
* `tf.random`, que permite construir un generador de valores aleatorio con el objetivo de crear tensores de diferentes tamaños.

In [5]:
# Creación de un tensor inicializado a ceros. # shape=[filas, columnas]
ceros = tf.zeros(shape=[3, 2], dtype=tf.int32)

# Creación de un tensor inicializado a unos.
# shape=[filas, columnas]
unos = tf.ones (shape=[4, 4], dtype=tf.float32)

# Creación de un tensor con valores aleatorios mediante semilla. 
# shape=[filas, columnas]
generador_1 = tf.random. Generator.from_seed (523434, alg='philox')
aleatorio_1 = generador_1.normal (shape=[3, 3])

print(aleatorio_1)

tf.Tensor(
[[ 0.71007586  1.2155863   0.9044451 ]
 [ 0.4229328   1.4938642   0.05864731]
 [-0.88086957 -1.7961171   0.23109847]], shape=(3, 3), dtype=float32)


### 3.3.3 - Concatenación de sensores

En algunas ocasiones, es necesario combinar la información de múltiples tensores, para ellos utilizamos la función `concat`:

In [6]:
A = tf.constant(
    [[1,1],
    [2,2]]
)
B = tf.constant(
    [[3,3],
    [4,4]]
)

# Concatenación de filas
AB_filas = tf.concat(values=[A,B], axis=1)
print(f"{AB_filas.numpy()}\n")

# Concatenación de columnas
AB_columnas = tf.concat(values=[A,B], axis=0)
print(AB_columnas.numpy())

[[1 1 3 3]
 [2 2 4 4]]

[[1 1]
 [2 2]
 [3 3]
 [4 4]]


### 3.3.4 - Redefinición de tamaño de tensores

En ciertas ocasiones, debemos realizar diferentes tipos de transformaciones sobre los tensores que implican modificar su tamaño (shape). Así, por ejemplo, si tenemos que transformar un tensor cuadrado de 2x2 en un tensor columna 1x4, podríamos transformar directamente el tamaño. Es muy importante tener en cuenta que los tamaños iniciales y finales deben ser complementarios.

In [7]:
inicial = tf.constant(
    [[1,2],
     [3,4]]
)
final = tf.reshape(tensor = inicial, shape = [1,4])

print(f"{inicial.numpy()}\n")
print(f"{final.numpy()}\n")

[[1 2]
 [3 4]]

[[1 2 3 4]]



### 3.3.5 - Transformando tensores en otro tipo de datos

En algunos casos resulta necesario transformar el tipo de datos almacenado en los tensores. Así por ejemplo, podemos reducir la precisión de la información de una matriz al transformar los valores reales en enteros. 

In [8]:
inicial = tf.constant(
    [[1.2, 3.4],[8.2, 1.3]], dtype=tf.float32)
final = tf.cast(inicial, tf.int32)

print(f"Tensor en format float: \n{inicial.numpy}\n")
print(f"Tensor en format int: \n{final.numpy}\n")

Tensor en format float: 
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1.2, 3.4],
       [8.2, 1.3]], dtype=float32)>>

Tensor en format int: 
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 3],
       [8, 1]])>>



## 3.4 - Cálculo de gradientes

Uno de los pasos más importantes a la hora de construir un proceso de aprendizaje es la generación de los diferentes gradientes que serán utilizados para el proceso de entrenamiento. Para poder extraer los gradientes de cualquier operación en TF, podemos utilizar GradientTape, que registra el gradiente de cualquier operación ejecutada bajo el tipo de gradiente. 

Así, si quisiéramos obtener el gradiente de la función "tangente hiperbólica" (una de las funciones de activación más utilizadas para la construcción de redes de neurones), deberíamos ejecutar el siguiente código:

$$
\text{tanh}(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}
$$

In [9]:
def my_function(x):
    return tf.tanh(x)

def get_gradient(input, activation_function):
    
    with tf.GradientTape() as gt:
        output = activation_function(input)
        
    return gt.gradient(output, input).numpy()

x = tf.Variable(1.5)
gradient = get_gradient(x, my_function)

print(f"{gradient} es el gradiente TANH para x={x.numpy()}")

0.18070673942565918 es el gradiente TANH para x=1.5


## 3.5 - Funciones

TF permite **ejecutar conjuntos de operaciones de manera eficiente y/o paralelizada** en diferentes tipos de dispositivos *hardware* utilizando **grafos de computación**. Los grafos son estructuras de datos que permiten la ejecución de **algoritmos construidos mediante dos componentes**:
1. Operaciones, que representan las diferentes unidades de cálculos que deben realizarse sobre la información
2. Tensores, que representan las distintas unidades de información que fluyen entre las operaciones.

Para poder crear un grafo mediante TF, debemos utilizar tf.function mediante una llamada directa o un decorador. Por ejemplo, en el siguiente código ejecutamos una capa convolucional:

In [10]:
import timeit

convolution_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def convolution_fn(image):
    return convolution_layer(image)

image = tf.zeros([1, 500, 500, 100])

# Ejecutamos la capa sin encapsular en una función
result_no_tf_fn = timeit.timeit(lambda: convolution_layer(image), number=12)

# Ejecutamos la capa encapsulada en una función
result_tf_fn = timeit.timeit(lambda: convolution_fn(image), number=12)

difference = result_no_tf_fn - result_tf_fn
print(f"Tiempo sin @tf.function: {result_no_tf_fn}")
print(f"Tiempo con @tf.function: {result_tf_fn}")
print(f"Diferencia de tiempos: {difference}")

Tiempo sin @tf.function: 3.2401181999593973
Tiempo con @tf.function: 2.798629099968821
Diferencia de tiempos: 0.4414890999905765


La diferencia entre las dos ejecuciones es muy pequeña. Sin embargo, si estamos entrenando un red neuronal donde esta ejecución se repite millones de veces, la diferencia de tiempo puede ser muy elevada.

## 3.6 - Operaciones matriciales

### 3.6.1 - Suma de matrices

In [11]:
A = tf.constant(
    [[3,7],
     [1,9]]
)
B = tf.constant(
    [[8,6],
     [1,5]]
)

tf.math.add(A,B)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[11, 13],
       [ 2, 14]])>

### 3.6.2 - Transposición de matrices

La trasposición de una matriz $A$ consiste en colocar sus filas en forma de columna respetando su orden

In [12]:
A = tf.constant(
    [[3,7],
     [1,9]]
)

tf.transpose(A)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 1],
       [7, 9]])>

### 3.6.3 - Multiplicación de matrices

La multiplicación de matrices es una operación que genera una nueva matriz cuyo tamaño estará definido por el número de filas de la matriz $A$ y el número de columnas de la matriz $B$. Esto implica que, para que dos matrices
puedan ser multiplicadas, el número de columnas de la matriz $A$ ha de ser igual al número de filas de la matriz $B$, siendo $A$ la primera matriz de la operación y $B$ la segunda.

In [13]:
A = tf.constant(
    [[3,7],
     [1,9]]
)
B = tf.constant(
    [[8,6],
     [1,5]]
)

tf.linalg.matmul(A,B)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[31, 53],
       [17, 51]])>

### 3.6.4 - Determinante de una matriz

El determinante de una matriz es una operación matemática que consiste en restar la multiplicación de los elementos de la diagonal principal a la multiplicación de los elementos de la diagonal secundaria.

In [14]:
A = tf.constant(
    [[3,21],
     [12,9]]
)
# El determinante debe tener un formato decimal
A = tf.dtypes.cast(A, tf.float32)

tf.linalg.det(A)

<tf.Tensor: shape=(), dtype=float32, numpy=-225.00002>

### 3.6.5 - Producto escalar de matrices

El producto escalar de matrices es una operación matemática que genera un valor escalar, es decir, un número a partir de dos matrices. Esto implica que, para que dos matrices puedan ser multiplicadas, el número de columnas de la matriz $A$ tiene que ser igual que el número de filas de la matriz $B$, siendo $A$ la primera matriz de la operación y $B$ la segunda.

In [15]:
A = tf.constant(
    [[3,7],
     [1,9]]
)
B = tf.constant(
    [[8,6],
     [1,5]]
)

tf.tensordot(a=A, b=B, axes=2)

<tf.Tensor: shape=(), dtype=int32, numpy=112>

### 3.6.6 - Matriz identidad

La matriz identidad es un tipo de matriz especial donde todos los valores son ceros ($0$), excepto los valores de ladiagonal principal, que son unos ($1$).

In [16]:
# Creamos una matriz de identidad cuadratica de tamaño (3,3)
tf.eye(num_rows = 3, num_columns = 3, dtype = tf.int32)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])>